In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
%pip install gdown

In [ ]:
import gdown
import shutil
url = 'https://drive.google.com/uc?export=download&id=1-mtINaXGw3whucSm8P9zMdQS8_qXvjAe'
output='/kaggle/working/data.zip'

gdown.download(url, output, quiet=False)
gdown.extractall(output, '/kaggle/working')
shutil.move('/kaggle/working/Seen Datasets','/kaggle/working/dataset')
shutil.os.remove(output)
# shutil.rmtree('/kaggle/working/dataset/Seen Datasets')

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, log_loss
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np

%matplotlib inline

In [ ]:
# Defining transformations
transform = transforms.Compose([
    transforms.Resize((416, 416)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
# Loading dataset
train_data = datasets.ImageFolder(
    root='/kaggle/working/dataset/train', 
    transform=transform
)

val_data = datasets.ImageFolder(
    root='/kaggle/working/dataset/val', 
    transform=transform
)

In [ ]:
# Hyperparameters
batch_size = 64
num_epochs = 60

# Data loaders
train_loader = DataLoader(
    train_data, 
    batch_size=batch_size, 
    shuffle=True, 
    num_workers=4
)

val_loader = DataLoader(
    val_data, 
    batch_size=batch_size, 
    shuffle=False, 
    num_workers=4
)

# **Model Architecture** 

In [ ]:
%pip install torch-summary

In [ ]:
class MyModel(nn.Module):
    def __init__(self, num_classes=25):
        super().__init__()
        self.features_extraction = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),  
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(kernel_size=3, stride=2),
              
            
            nn.Conv2d(64, 192, kernel_size=5, padding=2),  
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(192),
            nn.MaxPool2d(kernel_size=3, stride=2),
            
            
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(384),
            
            nn.Conv2d(384, 256, kernel_size=3, padding=1),  
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(256),
            
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(256),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.AdaptiveAvgPool2d((6, 6)),  
        )
        
        self.classifier = nn.Sequential(
            nn.Dropout(0.1),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(4096),
            
            nn.Dropout(0.1),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(4096),
            
            nn.Dropout(0.1),
            nn.Linear(4096, 512),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(512),
            
            nn.Linear(512, num_classes),
        )

    def forward(self, x):
        x = self.features_extraction(x)
        x = x.view(x.size(0), -1)
        return self.classifier(x)
    

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model = MyModel(num_classes=25)
model = torch.nn.DataParallel(model, device_ids = [0,1]).to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()  # For multi-class classification
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Using Adam optimizer

In [ ]:
# Lists to store metrics
train_metrics = {'accuracy': [], 'precision': [], 'recall': [], 'f1': [], 'roc_auc': [], 'log_loss': []}
val_metrics = {'accuracy': [], 'precision': [], 'recall': [], 'f1': [], 'roc_auc': [], 'log_loss': []}

In [ ]:
# Training loop
for epoch in range(num_epochs):
    model.train()
    train_preds = []
    train_probs = []
    train_labels = []
    running_loss = 0.0
    
    # Training progress bar
    for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs} - Training", unit='batch'):
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        train_preds.extend(predicted.cpu().numpy())
        train_labels.extend(labels.cpu().numpy())
        train_probs.extend(nn.functional.softmax(outputs.detach(), dim=1).cpu().numpy())
    
    # Calculate metrics for training
    train_accuracy = accuracy_score(train_labels, train_preds)
    train_precision = precision_score(train_labels, train_preds, average='macro')
    train_recall = recall_score(train_labels, train_preds, average='macro')
    train_f1 = f1_score(train_labels, train_preds, average='macro')
    train_roc_auc = roc_auc_score(train_labels, np.array(train_probs), multi_class='ovr')
    train_log_loss = log_loss(train_labels, np.array(train_probs))
    
    train_metrics['accuracy'].append(train_accuracy)
    train_metrics['precision'].append(train_precision)
    train_metrics['recall'].append(train_recall)
    train_metrics['f1'].append(train_f1)
    train_metrics['roc_auc'].append(train_roc_auc)
    train_metrics['log_loss'].append(train_log_loss)
    
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}, Accuracy: {train_accuracy:.2f}')
    
    model.eval()
    val_preds = []
    val_probs = []
    val_labels = []
    val_loss = 0.0
    
    # Validation progress bar
    for inputs, labels in tqdm(val_loader, desc=f"Epoch {epoch + 1}/{num_epochs} - Validation", unit='batch'):
        inputs, labels = inputs.to(device), labels.to(device)
        
        with torch.no_grad():
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            val_loss += loss.item()
            _, predicted = outputs.max(1)
            val_preds.extend(predicted.cpu().numpy())
            val_labels.extend(labels.cpu().numpy())
            val_probs.extend(nn.functional.softmax(outputs.detach(), dim=1).cpu().numpy())
    
    # Calculate metrics for validation
    val_accuracy = accuracy_score(val_labels, val_preds)
    val_precision = precision_score(val_labels, val_preds, average='macro')
    val_recall = recall_score(val_labels, val_preds, average='macro')
    val_f1 = f1_score(val_labels, val_preds, average='macro')
    val_roc_auc = roc_auc_score(val_labels, np.array(val_probs), multi_class='ovr')
    val_log_loss = log_loss(val_labels, np.array(val_probs))
    
    val_metrics['accuracy'].append(val_accuracy)
    val_metrics['precision'].append(val_precision)
    val_metrics['recall'].append(val_recall)
    val_metrics['f1'].append(val_f1)
    val_metrics['roc_auc'].append(val_roc_auc)
    val_metrics['log_loss'].append(val_log_loss)
    
    print(f'Validation Accuracy: {val_accuracy:.2f}')
    print(f'Validation Precision: {val_precision:.2f}')
    print(f'Validation Recall: {val_recall:.2f}')
    print(f'Validation F1 Score: {val_f1:.2f}')
    print(f'Validation ROC-AUC: {val_roc_auc:.2f}')
    print(f'Validation Log Loss: {val_log_loss:.2f}')

print('Training complete.')


In [ ]:
epochs_range = range(1, len(train_metrics['accuracy']) + 1)

plt.figure(figsize=(14, 10))
plt.suptitle('Performance Metrics')
# Accuracy
plt.subplot(3, 2, 1)
plt.plot(epochs_range, train_metrics['accuracy'], label='Training Accuracy', color='blue')
plt.plot(epochs_range, val_metrics['accuracy'], label='Validation Accuracy', color='orange')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Accuracy')
plt.legend()

# Precision
plt.subplot(3, 2, 2)
plt.plot(epochs_range, train_metrics['precision'], label='Training Precision', color='blue')
plt.plot(epochs_range, val_metrics['precision'], label='Validation Precision', color='orange')
plt.xlabel('Epochs')
plt.ylabel('Precision')
plt.title('Precision')
plt.legend()

# Recall
plt.subplot(3, 2, 3)
plt.plot(epochs_range, train_metrics['recall'], label='Training Recall', color='blue')
plt.plot(epochs_range, val_metrics['recall'], label='Validation Recall', color='orange')
plt.xlabel('Epochs')
plt.ylabel('Recall')
plt.title('Recall')
plt.legend()

# F1 Score
plt.subplot(3, 2, 4)
plt.plot(epochs_range, train_metrics['f1'], label='Training F1 Score', color='blue')
plt.plot(epochs_range, val_metrics['f1'], label='Validation F1 Score', color='orange')
plt.xlabel('Epochs')
plt.ylabel('F1 Score')
plt.title('F1 Score')
plt.legend()

# ROC-AUC
plt.subplot(3, 2, 5)
plt.plot(epochs_range, train_metrics['roc_auc'], label='Training ROC-AUC', color='blue')
plt.plot(epochs_range, val_metrics['roc_auc'], label='Validation ROC-AUC', color='orange')
plt.xlabel('Epochs')
plt.ylabel('ROC-AUC')
plt.title('ROC-AUC')
plt.legend()

# Log Loss
plt.subplot(3, 2, 6)
plt.plot(epochs_range, train_metrics['log_loss'], label='Training Log Loss', color='blue')
plt.plot(epochs_range, val_metrics['log_loss'], label='Validation Log Loss', color='orange')
plt.xlabel('Epochs')
plt.ylabel('Log Loss')
plt.title('Log Loss')
plt.legend()


plt.tight_layout()
plt.savefig('/kaggle/working/result.jpg')
plt.show()

In [ ]:
from torchsummary import summary

_ = summary(model)

In [ ]:
torch.save(model, "model.pth")